# IMDB 영화평 감성분석

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('./data/labeledTrainData.tsv',
                header = 0, sep = '\t', quoting = 3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
# <br /> 태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />','')

In [6]:
# 영어 이외 문자 공백으로 변환
import re
df['review'] = df.review.apply(lambda x: re.sub('[^a-zA-Z]', '', x))

In [7]:
from sklearn.model_selection import train_test_split


feature_df = df.drop(['id','sentiment'], axis=1, inplace = False)
X_train, X_test, y_train, y_test = train_test_split(
    feature_df.review, df.sentiment, test_size = 0.3, random_state = 156
)
# X_train.shape, X_test.shape
# X_train = X_train.reshape(-1,1)
# X_test = X_test.reshape(-1,1)
X_train.shape, X_test.shape

((17500,), (7500,))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- CountVectorizer

In [9]:
cvect = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect.fit(X_train)
X_train_count = cvect.transform(X_train)
X_test_count = cvect.transform(X_test)

In [10]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_count, y_train)
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred)

0.4918666666666667

- TfidfVectorizer

In [11]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

In [12]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf, y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.4918666666666667

In [13]:
import joblib
joblib.dump(tfidf_vect, './model/imdb_vect.pkl')
joblib.dump(lr_clf, './model/imdb_ir.pkl')

['./model/imdb_ir.pkl']

In [14]:
new_vect = joblib.load('./model/imdb_vect.pkl')
new_lr = joblib.load('./model/imdb_ir.pkl')

In [15]:
new_X_test = cvect.transform(X_test)

In [16]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.4918666666666667

### Pipeline 저장 가능?

In [17]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range = (1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + LogisticRegression 정확도 : {acc:.4f}')

Count Vectorizer + LogisticRegression 정확도 : 0.4919


In [18]:
joblib.dump(pipeline, 'model/pipeline.pkl')

['model/pipeline.pkl']

In [19]:
new_pipe = joblib.load('./model/pipeline.pkl')